In [1]:
import numpy as np 
import pandas as pd
import csv

In [2]:
#!pip install fasttext
import fasttext

In [4]:
#!pip install gensim
from gensim.utils import simple_preprocess

Для обучения не забудьте перенести в папку с этим .ipynb-файлом parquet-файлы полученные в "01 Preparation" (train_data и test_data)

In [3]:
train_data = pd.read_parquet('train_data.parquet')

In [4]:
train_data.head()

,category_id,Docs
0,12171,кабели зарядный кабель borofone bx1 lightning ...
1,14233,трусы трусы sela трусы слипы из эластичного бе...
3,13429,краски пигменты гуашь юный волшебник 12 цв...
4,2789,колбы колба для кальяна крафт разные цвета у...
5,12834,пижамы пижама женская однотонная с шортами лё...


In [5]:
#Несбалансированная выборка
train_data['category_id'].value_counts().describe()

count     874.000000
mean      104.256293
std       300.107191
min         1.000000
25%        14.000000
50%        38.000000
75%        99.000000
max      6590.000000
Name: category_id, dtype: float64

In [7]:
#Подготовка к FastText
train_data.Docs = train_data.Docs.apply(lambda x: ' '.join(simple_preprocess(x)))
train_data.category_id = train_data.category_id.apply(lambda x: '__label__' + str(x))

In [8]:
#Размешаем данные
#у нас в выборке 100000 объектов. Возьмем 5000 чтобы быстрее обучать
train_data = train_data.sample(frac=1, random_state=17)[:5000]

In [9]:
#Разбиваем на train/val
n = int(0.8*len(train_data))
train_df = train_data[:n]
val_df = train_data[n:]

In [10]:
# сохраняем csv в txt. Это важно для FastText
train_df[['Docs', 'category_id']].to_csv('fasttext_train.txt',
                index = False, sep = ' ', header = None,
                quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")

val_df[['Docs', 'category_id']].to_csv('fasttext_val.txt', index = False, sep = ' ', header = None, 
              quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")

In [ ]:
'''
ИЗ ПРЕДЫДУЩЕГО ПЕРЕБОБРА ПАРАМЕТРОВ
********CURRENT BEST PARAMS*******
0.83298755186722
lr =  2.0
dim =  40
ws =  3
epoch =  70
neg =  5
wordNgrams =  3
******************

ВЫВОД:
1) увеличение эпох обучения до 70 положительно сказывается на f1
2) увеличение lr положительно сказывается на f1
dim 40
ws = 3
neg =  3
wordNgrams =  3
'''

In [13]:
epochs_list = [30, 50, 70]
dim_lst =  [30, 40, 50]
neg_lst = [3, 5]
ws_lst = [3]
wordNgrams_lst = [3]
lr_lst = [0.5, 0.7, 1.0, 2.0]

In [14]:
val_f1 = 0
for epochs in epochs_list:
    print('Epochs: ', epochs)
    for dim in dim_lst:
        print('* dim: ', dim)
        for neg in neg_lst:
            print('** neg: ', neg)
            for ws in ws_lst:
                print('*** ws = ', ws)
                for w in wordNgrams_lst:
                    print('**** wordNgrams = ', w)
                    for lr in lr_lst:
                        print('***** lr = ', lr)
                        model = fasttext.train_supervised('fasttext_train.txt',
                                                          lr=lr, 
                                                          dim=dim, 
                                                          ws=ws,
                                                          epoch=epochs,
                                                          neg=neg,
                                                          wordNgrams=w)
                        
                        _, precision, recall = model.test('fasttext_val.txt')
                        
                        val_cur = (2*precision*recall) / (precision+recall)
                        if (val_cur > val_f1):
                            val_f1 = val_cur
                            print('********CURRENT BEST PARAMS*******')
                            print(val_f1)
                            print('lr = ', lr)
                            print('dim = ', dim)
                            print('ws = ', ws)
                            print('epoch = ', epochs)
                            print('neg = ', neg)
                            print('wordNgrams = ', w)
                            print('******************')
                            print(' ')

Epochs:  30
* dim:  30
** neg:  3
*** ws =  3
**** wordNgrams =  3
***** lr =  0.5
********CURRENT BEST PARAMS*******
0.7282157676348547
lr =  0.5
dim =  30
ws =  3
epoch =  30
neg =  3
wordNgrams =  3
******************
 
***** lr =  0.7
********CURRENT BEST PARAMS*******
0.779045643153527
lr =  0.7
dim =  30
ws =  3
epoch =  30
neg =  3
wordNgrams =  3
******************
 
***** lr =  1.0
********CURRENT BEST PARAMS*******
0.7966804979253111
lr =  1.0
dim =  30
ws =  3
epoch =  30
neg =  3
wordNgrams =  3
******************
 
***** lr =  2.0
********CURRENT BEST PARAMS*******
0.821576763485477
lr =  2.0
dim =  30
ws =  3
epoch =  30
neg =  3
wordNgrams =  3
******************
 
** neg:  5
*** ws =  3
**** wordNgrams =  3
***** lr =  0.5
***** lr =  0.7
***** lr =  1.0
***** lr =  2.0
********CURRENT BEST PARAMS*******
0.8257261410788381
lr =  2.0
dim =  30
ws =  3
epoch =  30
neg =  5
wordNgrams =  3
******************
 
* dim:  40
** neg:  3
*** ws =  3
**** wordNgrams =  3
***** lr